In [1]:
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow-federated-nightly
!pip install --quiet --upgrade nest-asyncio
!pip install --quiet --upgrade tb-nightly  # or tensorboard, but not both

import nest_asyncio
nest_asyncio.apply()

Found existing installation: tb-nightly 2.7.0a20211013
Uninstalling tb-nightly-2.7.0a20211013:
  Successfully uninstalled tb-nightly-2.7.0a20211013


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\nest_asyncio.py'
Consider using the `--user` option or check the permissions.

ERROR: Invalid requirement: '#'


In [2]:
import collections
import pickle
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import random
np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [9]:
def flipping_client_taken(client_taken,client_percent, data_percent):
    client_data={}
    initial='clients'
    random_client = random.sample(range(0,client_taken), round(client_taken*client_percent))
    random_client = np.sort(random_client)
    print(random_client)
    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
    x=0
    y=0
    for i in range(client_taken):
        client_dataset = emnist_train.create_tf_dataset_for_client(
          emnist_train.client_ids[i])
        client_name=['{}_{}'.format(initial, i+1)]
        bla=np.zeros((1,784))
        sla= np.zeros((1,1))
        if random_client[y]==x:
            print("engaged")
            data_lenght=len(client_dataset)
            random_data= random.sample(range(0,data_lenght), round(data_lenght*data_percent))
            random_data= np.sort(random_data)
#             print(random_data)
            m=0
            n=0
            fake_label= random.randrange(0,9,1)
            for example in client_dataset:
            # Append counts individually per label to make plots
            # more colorful instead of one color per plot.
                label = example['label'].numpy()
                label= label.reshape(-1,1)
                label= np.array(label)
#                 print(i,label,j)
                if random_data[n]==m:
                    label=[[fake_label]]
#                     label= label.reshape((-1,1))
                    label= np.array(label)
#                     print(label)
                    sla= np.concatenate((sla,label))
                    if n<len(random_data)-1:
                        n= n+1
#                     print(j)
                else:
                    sla= np.concatenate((sla,label))
                data = example['pixels'].numpy()
                data = data.reshape(1,784)
                bla= np.concatenate((bla,data))
                m+=1
            if y<len(random_client)-1:
                y= y+1
        else:
            print('non_engaed')
            for example in client_dataset:
        # Append counts individually per label to make plots
        # more colorful instead of one color per plot.
                label = example['label'].numpy()
                label= label.reshape(-1,1)
                label= np.array(label)
                sla= np.concatenate((sla,label))
                data = example['pixels'].numpy()
                data = data.reshape(1,784)
                bla= np.concatenate((bla,data))
        x= x+1
        print(x,y)

                

        bla= bla[1:len(bla),:]
        sla= sla[1:len(sla),:]
#         sla= tf.keras.utils.to_categorical(sla, 10)
        dataset= list(zip(bla,sla))
        present_data= {client_name[0]:dataset}
        client_data.update(present_data)
    return client_data, random_client

In [10]:
def test_client_taken(client_taken):
    bla=np.zeros((1,784))
    sla= np.zeros((1,1))
    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
    for i in range(client_taken):
        client_dataset = emnist_test.create_tf_dataset_for_client(
          emnist_test.client_ids[i])
        
        for example in client_dataset:
        # Append counts individually per label to make plots
        # more colorful instead of one color per plot.
            label = example['label'].numpy()
            label= label.reshape(-1,1)
            label= np.array(label)
            sla= np.concatenate((sla,label))
            data = example['pixels'].numpy()
            data = data.reshape(1,784)
            bla= np.concatenate((bla,data))

    bla= bla[1:len(bla),:]
    sla= sla[1:len(sla),:]
#     sla= tf.keras.utils.to_categorical(sla, 10)
    return bla, sla

In [11]:
def save_file(file_name,data):
    open_file = open(file_name, "wb")
    pickle.dump(data, open_file)
    open_file.close()


In [22]:
#train data creation
num_client=3000
test_client= 3000
client_percent= .1
data_percent= 1
client_data, bad_client= flipping_client_taken(num_client, client_percent,data_percent)


[   6   13   18   21   40   59   60   62   71   74   85  108  120  139
  147  152  154  157  192  197  198  201  229  276  291  303  351  364
  367  373  377  388  397  398  403  423  428  433  439  447  459  460
  481  485  506  514  517  526  537  538  565  586  610  624  632  635
  646  655  666  667  678  687  688  702  704  709  723  730  739  772
  781  784  804  806  839  855  861  888  892  897  898  902  918  922
  958  969  978  984 1005 1008 1021 1029 1046 1063 1069 1070 1077 1079
 1088 1092 1093 1097 1134 1135 1164 1169 1176 1189 1191 1192 1208 1209
 1217 1238 1243 1250 1262 1269 1273 1276 1288 1293 1294 1296 1301 1319
 1322 1330 1333 1349 1351 1354 1369 1370 1378 1383 1398 1406 1410 1423
 1431 1433 1441 1446 1455 1495 1501 1502 1505 1532 1547 1584 1585 1594
 1597 1603 1622 1627 1629 1641 1642 1654 1669 1674 1677 1683 1736 1754
 1760 1770 1789 1792 1796 1809 1811 1815 1832 1840 1848 1853 1857 1858
 1866 1871 1875 1910 1911 1924 1930 1946 1948 1956 1959 1961 1965 1968
 1971 

non_engaed
396 32
non_engaed
397 32
engaged
398 33
engaged
399 34
non_engaed
400 34
non_engaed
401 34
non_engaed
402 34
non_engaed
403 34
engaged
404 35
non_engaed
405 35
non_engaed
406 35
non_engaed
407 35
non_engaed
408 35
non_engaed
409 35
non_engaed
410 35
non_engaed
411 35
non_engaed
412 35
non_engaed
413 35
non_engaed
414 35
non_engaed
415 35
non_engaed
416 35
non_engaed
417 35
non_engaed
418 35
non_engaed
419 35
non_engaed
420 35
non_engaed
421 35
non_engaed
422 35
non_engaed
423 35
engaged
424 36
non_engaed
425 36
non_engaed
426 36
non_engaed
427 36
non_engaed
428 36
engaged
429 37
non_engaed
430 37
non_engaed
431 37
non_engaed
432 37
non_engaed
433 37
engaged
434 38
non_engaed
435 38
non_engaed
436 38
non_engaed
437 38
non_engaed
438 38
non_engaed
439 38
engaged
440 39
non_engaed
441 39
non_engaed
442 39
non_engaed
443 39
non_engaed
444 39
non_engaed
445 39
non_engaed
446 39
non_engaed
447 39
engaged
448 40
non_engaed
449 40
non_engaed
450 40
non_engaed
451 40
non_engaed
452 4

non_engaed
861 76
engaged
862 77
non_engaed
863 77
non_engaed
864 77
non_engaed
865 77
non_engaed
866 77
non_engaed
867 77
non_engaed
868 77
non_engaed
869 77
non_engaed
870 77
non_engaed
871 77
non_engaed
872 77
non_engaed
873 77
non_engaed
874 77
non_engaed
875 77
non_engaed
876 77
non_engaed
877 77
non_engaed
878 77
non_engaed
879 77
non_engaed
880 77
non_engaed
881 77
non_engaed
882 77
non_engaed
883 77
non_engaed
884 77
non_engaed
885 77
non_engaed
886 77
non_engaed
887 77
non_engaed
888 77
engaged
889 78
non_engaed
890 78
non_engaed
891 78
non_engaed
892 78
engaged
893 79
non_engaed
894 79
non_engaed
895 79
non_engaed
896 79
non_engaed
897 79
engaged
898 80
engaged
899 81
non_engaed
900 81
non_engaed
901 81
non_engaed
902 81
engaged
903 82
non_engaed
904 82
non_engaed
905 82
non_engaed
906 82
non_engaed
907 82
non_engaed
908 82
non_engaed
909 82
non_engaed
910 82
non_engaed
911 82
non_engaed
912 82
non_engaed
913 82
non_engaed
914 82
non_engaed
915 82
non_engaed
916 82
non_engaed

1302 125
non_engaed
1303 125
non_engaed
1304 125
non_engaed
1305 125
non_engaed
1306 125
non_engaed
1307 125
non_engaed
1308 125
non_engaed
1309 125
non_engaed
1310 125
non_engaed
1311 125
non_engaed
1312 125
non_engaed
1313 125
non_engaed
1314 125
non_engaed
1315 125
non_engaed
1316 125
non_engaed
1317 125
non_engaed
1318 125
non_engaed
1319 125
engaged
1320 126
non_engaed
1321 126
non_engaed
1322 126
engaged
1323 127
non_engaed
1324 127
non_engaed
1325 127
non_engaed
1326 127
non_engaed
1327 127
non_engaed
1328 127
non_engaed
1329 127
non_engaed
1330 127
engaged
1331 128
non_engaed
1332 128
non_engaed
1333 128
engaged
1334 129
non_engaed
1335 129
non_engaed
1336 129
non_engaed
1337 129
non_engaed
1338 129
non_engaed
1339 129
non_engaed
1340 129
non_engaed
1341 129
non_engaed
1342 129
non_engaed
1343 129
non_engaed
1344 129
non_engaed
1345 129
non_engaed
1346 129
non_engaed
1347 129
non_engaed
1348 129
non_engaed
1349 129
engaged
1350 130
non_engaed
1351 130
engaged
1352 131
non_engae

1727 166
non_engaed
1728 166
non_engaed
1729 166
non_engaed
1730 166
non_engaed
1731 166
non_engaed
1732 166
non_engaed
1733 166
non_engaed
1734 166
non_engaed
1735 166
non_engaed
1736 166
engaged
1737 167
non_engaed
1738 167
non_engaed
1739 167
non_engaed
1740 167
non_engaed
1741 167
non_engaed
1742 167
non_engaed
1743 167
non_engaed
1744 167
non_engaed
1745 167
non_engaed
1746 167
non_engaed
1747 167
non_engaed
1748 167
non_engaed
1749 167
non_engaed
1750 167
non_engaed
1751 167
non_engaed
1752 167
non_engaed
1753 167
non_engaed
1754 167
engaged
1755 168
non_engaed
1756 168
non_engaed
1757 168
non_engaed
1758 168
non_engaed
1759 168
non_engaed
1760 168
engaged
1761 169
non_engaed
1762 169
non_engaed
1763 169
non_engaed
1764 169
non_engaed
1765 169
non_engaed
1766 169
non_engaed
1767 169
non_engaed
1768 169
non_engaed
1769 169
non_engaed
1770 169
engaged
1771 170
non_engaed
1772 170
non_engaed
1773 170
non_engaed
1774 170
non_engaed
1775 170
non_engaed
1776 170
non_engaed
1777 170
non

engaged
2152 220
non_engaed
2153 220
engaged
2154 221
non_engaed
2155 221
non_engaed
2156 221
non_engaed
2157 221
engaged
2158 222
non_engaed
2159 222
non_engaed
2160 222
non_engaed
2161 222
engaged
2162 223
non_engaed
2163 223
engaged
2164 224
non_engaed
2165 224
engaged
2166 225
non_engaed
2167 225
non_engaed
2168 225
non_engaed
2169 225
non_engaed
2170 225
non_engaed
2171 225
non_engaed
2172 225
non_engaed
2173 225
non_engaed
2174 225
non_engaed
2175 225
non_engaed
2176 225
non_engaed
2177 225
non_engaed
2178 225
non_engaed
2179 225
non_engaed
2180 225
non_engaed
2181 225
non_engaed
2182 225
non_engaed
2183 225
non_engaed
2184 225
non_engaed
2185 225
non_engaed
2186 225
non_engaed
2187 225
non_engaed
2188 225
non_engaed
2189 225
non_engaed
2190 225
non_engaed
2191 225
non_engaed
2192 225
non_engaed
2193 225
non_engaed
2194 225
engaged
2195 226
engaged
2196 227
non_engaed
2197 227
engaged
2198 228
non_engaed
2199 228
non_engaed
2200 228
non_engaed
2201 228
non_engaed
2202 228
non_eng

non_engaed
2572 259
non_engaed
2573 259
engaged
2574 260
non_engaed
2575 260
non_engaed
2576 260
non_engaed
2577 260
non_engaed
2578 260
non_engaed
2579 260
non_engaed
2580 260
non_engaed
2581 260
non_engaed
2582 260
non_engaed
2583 260
non_engaed
2584 260
non_engaed
2585 260
non_engaed
2586 260
non_engaed
2587 260
non_engaed
2588 260
non_engaed
2589 260
non_engaed
2590 260
non_engaed
2591 260
engaged
2592 261
non_engaed
2593 261
non_engaed
2594 261
non_engaed
2595 261
non_engaed
2596 261
non_engaed
2597 261
non_engaed
2598 261
non_engaed
2599 261
non_engaed
2600 261
non_engaed
2601 261
non_engaed
2602 261
non_engaed
2603 261
non_engaed
2604 261
non_engaed
2605 261
engaged
2606 262
non_engaed
2607 262
non_engaed
2608 262
non_engaed
2609 262
non_engaed
2610 262
non_engaed
2611 262
non_engaed
2612 262
engaged
2613 263
non_engaed
2614 263
engaged
2615 264
non_engaed
2616 264
non_engaed
2617 264
non_engaed
2618 264
engaged
2619 265
non_engaed
2620 265
non_engaed
2621 265
non_engaed
2622 26

2996 299
non_engaed
2997 299
non_engaed
2998 299
non_engaed
2999 299
non_engaed
3000 299


In [23]:
# test data creation
# test_data, test_label= test_client_taken(num_client)

In [24]:
file_name = f"Dataset_flip_{num_client}_{client_percent}_{data_percent}.pkl"

In [25]:
data=[client_data,bad_client]

In [26]:
save_file(file_name,data)